### Group ID: IR GROUP 73
### Group Members Name with Student ID:
1.RATNABABU MAMIDI - 2023AB05147<br>
2.Gujja Anirudh - 2023AA05236<br>
3.GUNTUKU ANANDA RAJ - 2023AB05189<br>
4.S SUJANA - 2023AB05187<br>


In [4]:
pip install nltk pandas PyPDF2 python-docx

Note: you may need to restart the kernel to use updated packages.


In [5]:
import os
import time
import nltk
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from docx import Document as docx_document
from PyPDF2 import PdfReader
import openpyxl

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\aniru\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aniru\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
import random

class SkipNode:
    def __init__(self, value, level):
        self.value = value
        self.forward = [None] * (level + 1)

class SkipList:
    def __init__(self, max_level, p):
        self.max_level = max_level
        self.p = p
        self.header = self.create_node(self.max_level, (-1, -1))
        self.level = 0

    def create_node(self, level, value):
        return SkipNode(value, level)

    def random_level(self):
        level = 0
        while random.random() < self.p and level < self.max_level:
            level += 1
        return level

    def insert(self, value):
        update = [None] * (self.max_level + 1)
        current = self.header
        for i in range(self.level, -1, -1):
            while current.forward[i] and current.forward[i].value < value:
                current = current.forward[i]
            update[i] = current
        current = current.forward[0]

        if current is None or current.value != value:
            level = self.random_level()
            if level > self.level:
                for i in range(self.level + 1, level + 1):
                    update[i] = self.header
                self.level = level
            new_node = self.create_node(level, value)
            for i in range(level + 1):
                new_node.forward[i] = update[i].forward[i]
                update[i].forward[i] = new_node

    def search(self, value):
        current = self.header
        for i in range(self.level, -1, -1):
            while current.forward[i] and current.forward[i].value < value:
                current = current.forward[i]
        current = current.forward[0]
        if current and current.value == value:
            return current
        return None


In [7]:
def read_docx(file_path):
    """
    Reads text from a .docx file.

    :param file_path: The path to the .docx file.
    :return: The full text of the document as a single string.
    """
    doc = docx_document(file_path)
    full_text = []
    for para in doc.paragraphs:
        full_text.append(para.text)
    return '\n'.join(full_text)

def read_pdf(file_path):
    """
    Reads text from a .pdf file.

    :param file_path: The path to the .pdf file.
    :return: The full text of the document as a single string.
    """
    with open(file_path, 'rb') as file:
        reader = PdfReader(file)
        full_text = []
        for page in reader.pages:
            full_text.append(page.extract_text())
    return '\n'.join(full_text)

def read_xlsx(file_path):
    """
    Reads text from a .xlsx file (assuming single sheet).

    :param file_path: The path to the .xlsx file.
    :return: The full text of the spreadsheet as a single string.
    """
    workbook = openpyxl.load_workbook(file_path)
    sheet = workbook.active
    full_text = []
    for row in sheet.iter_rows(values_only=True):
        for cell in row:
            if cell:
                full_text.append(str(cell))
    return '\n'.join(full_text)

def read_txt(file_path):
    """
    Reads text from a .txt file.

    :param file_path: The path to the .txt file.
    :return: The full text of the document as a single string.
    """
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

def preprocess_text(text):
    """
    Preprocesses text by tokenizing, removing stop words, and stemming.

    :param text: The raw text to preprocess.
    :return: A list of preprocessed tokens.
    """
    tokens = word_tokenize(text.lower())
    tokens = [token for token in tokens if token not in string.punctuation]
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]
    stemmer = PorterStemmer()
    stemmed_tokens = [stemmer.stem(token) for token in tokens]
    return stemmed_tokens

def process_documents(directory):
    """
    Processes all documents in a directory to construct a positional index.

    :param directory: The path to the directory containing the documents.
    :return: A positional index for all documents in the directory.
    """
    positional_index = {}
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        if filename.endswith('.docx'):
            text = read_docx(file_path)
        elif filename.endswith('.pdf'):
            text = read_pdf(file_path)
        elif filename.endswith('.xlsx'):
            text = read_xlsx(file_path)
        elif filename.endswith('.txt'):
            text = read_txt(file_path)
        else:
            continue
        tokens = preprocess_text(text)
        doc_id = filename
        for position, token in enumerate(tokens):
            if token not in positional_index:
                positional_index[token] = SkipList(16, 0.5)
            positional_index[token].insert((doc_id, position))
    return positional_index

# Specifying the directory containing the documents
documents_directory = r'C:\Users\aniru\OneDrive\Desktop\IR Corpus'

# Processing the documents to create the positional index
positional_index = process_documents(documents_directory)


In [8]:
import time
import random
import string

def search_inverted_index(positional_index, term):
    """
    Searches for a term in the basic inverted index.

    :param positional_index: The positional index.
    :param term: The term to search for.
    :return: The postings list for the term if found, otherwise None.
    """
    if term in positional_index:
        return positional_index[term]
    return None

def search_skip_list(positional_index, term):
    """
    Searches for a term in the skip list-enhanced index.

    :param positional_index: The positional index.
    :param term: The term to search for.
    :return: The postings list for the term if found, otherwise None.
    """
    if term in positional_index:
        skip_list = positional_index[term]
        results = []
        current = skip_list.header.forward[0]
        while current:
            if current.value[0] == term:
                results.append(current.value)
            current = current.forward[0]
        return results
    return None

def compare_search_performance(queries, positional_index):
    """
    Compares the search performance between the basic inverted index and the skip list-enhanced index.

    :param queries: A list of query terms.
    :param positional_index: The positional index.
    """
    basic_times = []
    skip_list_times = []

    for query in queries:
        # Measuring search time for basic inverted index
        start = time.perf_counter()
        search_inverted_index(positional_index, query)
        end = time.perf_counter()
        basic_times.append(end - start)

        # Measuring search time for skip list-enhanced index
        start = time.perf_counter()
        search_skip_list(positional_index, query)
        end = time.perf_counter()
        skip_list_times.append(end - start)

    # Calculating average search times
    avg_basic_time = sum(basic_times) / len(basic_times)
    avg_skip_list_time = sum(skip_list_times) / len(skip_list_times)

    # Printing average search times
    print(f"Average search time in basic inverted index: {avg_basic_time:.10f} seconds")
    print(f"Average search time in skip list-enhanced index: {avg_skip_list_time:.10f} seconds")

# Sample queries for performance comparison
queries = ['independence', 'tackled', 'clay', 'Chola', 'beads']

# Comparing search performance
compare_search_performance(queries, positional_index)


Average search time in basic inverted index: 0.0000007200 seconds
Average search time in skip list-enhanced index: 0.0000010400 seconds


In [9]:
def wildcard_query(positional_index, start_char, end_char):
    """
    Performs a wildcard query on the positional index to find terms that start with a specific character and end with another specific character.

    :param positional_index: The positional index containing terms and their postings lists.
    :param start_char: The character that the terms should start with.
    :param end_char: The character that the terms should end with.
    :return: A list of terms that match the wildcard query.
    """
    results = []
    for term in positional_index:
        if term.startswith(start_char) and term.endswith(end_char):
            results.append(term)
    return results

# Performing a wildcard query for terms that start with 't' and end with 'e'
wildcard_results = wildcard_query(positional_index, 't', 'e')
print(f"Wildcard query results: {wildcard_results}")


Wildcard query results: ['throne', 'trade', 'tree', 'theme', 'trace', 'take', 'time', 'thrive', 'te', 'tale']
